# **Finetuning**  


## **Finetuning Overview**  


## **When to Finetune**  


### Reasons to Finetune  


### Reasons Not to Finetune  


### Finetuning and RAG  


## **Memory Bottlenecks**  


### Backpropagation and Trainable Parameters  


### Memory Math  


### Numerical Representations  


### Quantization  


## **Finetuning Techniques**  


### Parameter-Efficient Finetuning  


### Model Merging and Multi-Task Finetuning  


### Finetuning Tactics  


## **Summary**  